# CytoTable (convert) and Pandas (Merges) Performance Comparisons

This notebook explores CytoTable (convert) and Pandas (DataFrame merges) usage with datasets of varying size to help describe performance impacts.

In [1]:
import itertools
import json
import os
import pathlib
import re
import signal
import subprocess
from datetime import datetime

import pandas as pd
import plotly.express as px
import plotly.io as pio
import psutil
from IPython.display import Image
from utilities import (
    get_memory_peak_and_time_duration,
    get_parsl_peak_memory,
    get_system_info,
)

# set plotly default theme
pio.templates.default = "simple_white"
# monitoring database for parsl multiprocessing work
db_file = "runinfo/monitoring.db"

In [2]:
# show the system information
_ = get_system_info(show_output=True)


System Information:
Operating System: Linux
Machine Type: x86_64
Processor: x86_64
CPU Cores (Logical): 16
CPU Cores (Physical): 8
Total RAM (GB): 62.68
Python Version: 3.11.9


In [3]:
# observe the virtual env for dependency inheritance with memray
# from subprocedure calls
"/".join(
    subprocess.run(
        [
            "which",
            "python",
        ],
        capture_output=True,
        check=True,
    )
    # decode bytestring as utf-8
    .stdout.decode("utf-8")
    # remove personal file structure
    .split("/")[6:]
    # replace final newline
).replace("\n", "")

'cytotable-benchmarks-2FApAEcz-py3.11/bin/python'

In [4]:
# target file or table names
image_dir = "images"
examples_dir = "examples"
join_read_time_image = (
    f"{image_dir}/cytotable-and-pandas-comparisons-join-completion-time-csv.png"
)
join_mem_size_image = (
    f"{image_dir}/cytotable-and-pandas-comparisons-join-memory-size-csv.png"
)
example_files_list = [
    f"{examples_dir}/cytotable_convert_examplehuman_multiprocess_csv.py",
    f"{examples_dir}/cytotable_convert_examplehuman_multithread_csv.py",
    f"{examples_dir}/pandas_merge_examplehuman_csv.py",
]
example_data_list = [
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x2",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x4",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x8",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x16",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x32",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x64",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x128",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x256",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x512",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x1024",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x2048",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x4096",
    f"{examples_dir}/data/examplehuman_cellprofiler_features_csv-x8192",
]

# format for memray time strings
tformat = "%Y-%m-%d %H:%M:%S.%f%z"

In [5]:
# Define the Parquet file path
results_parquet_file = "cytotable_pandas_results.parquet"

# Load existing results if available
if pathlib.Path(results_parquet_file).exists():
    df_results = pd.read_parquet(results_parquet_file)
    results = df_results.to_dict(orient="records")
else:
    results = []

In [6]:
# Number of iterations for each combination
num_iterations = 6

# Loop through each combination of example file and data file
for example_file, example_data in itertools.product(
    example_files_list, example_data_list
):
    for iteration in range(num_iterations):

        print(f"Starting {example_file} with {example_data}, iteration {iteration}.")
        # Skip if this combination and iteration are already processed
        if any(
            result["file_input"] == example_file
            and result["data_input"] == example_data
            and result["iteration"] == iteration
            for result in results
        ):
            print(
                f"Skipping already processed: {example_file} with {example_data}, iteration {iteration}"
            )
            continue

        try:
            # gather memory peak and time duration
            memory_peak, time_duration = get_memory_peak_and_time_duration(
                cmd=[
                    "python",
                    example_file,
                    example_data,
                ],
                polling_pause_seconds=0.1,
                # if we have a multiprocessed parsl process skip memory
                # (we will check this via parsl monitoring).
                skip_memory_check=("multiprocess" in example_file),
            )

            # Append the result
            results.append(
                {
                    "file_input": example_file.replace(f"{examples_dir}/", ""),
                    "data_input": example_data,
                    "iteration": iteration,
                    "time_duration (secs)": time_duration,
                    "peak_memory (bytes)": (
                        memory_peak
                        # if we have a multiprocessed parsl result we must
                        # gather the peak memory using parsl's monitoring
                        # database.
                        if "multiprocess" not in example_file
                        else get_parsl_peak_memory(db_file=db_file)
                    ),
                }
            )

            # Save intermediate results to Parquet
            df_results = pd.DataFrame(results)
            df_results.to_parquet(results_parquet_file, index=False)

        except Exception as e:
            print(
                f"Error processing {example_file} with {example_data}, iteration {iteration}: {e}"
            )

        finally:
            # remove monitoring database if present from parsl processing
            if pathlib.Path(db_file).is_file():
                pathlib.Path(db_file).unlink()
            print(
                f"Finished {example_file} with {example_data}, iteration {iteration}."
            )


# Final save to Parquet
df_results = pd.DataFrame(results)
df_results.to_parquet(results_parquet_file, index=False)

print(f"Processing complete. Results saved to '{results_parquet_file}'.")

Starting examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 0.


Finished examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 0.
Starting examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 1.


Finished examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 1.
Starting examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 2.


Finished examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 2.
Starting examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 3.


Finished examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 3.
Starting examples/cytotable_convert_examplehuman_multiprocess_csv.py with examples/data/examplehuman_cellprofiler_features_csv, iteration 4.


...

Finished examples/pandas_merge_examplehuman_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x8192, iteration 1.
Starting examples/pandas_merge_examplehuman_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x8192, iteration 2.


Finished examples/pandas_merge_examplehuman_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x8192, iteration 2.
Starting examples/pandas_merge_examplehuman_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x8192, iteration 3.


Finished examples/pandas_merge_examplehuman_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x8192, iteration 3.
Starting examples/pandas_merge_examplehuman_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x8192, iteration 4.


Finished examples/pandas_merge_examplehuman_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x8192, iteration 4.
Starting examples/pandas_merge_examplehuman_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x8192, iteration 5.


Finished examples/pandas_merge_examplehuman_csv.py with examples/data/examplehuman_cellprofiler_features_csv-x8192, iteration 5.
Processing complete. Results saved to 'cytotable_pandas_results.parquet'.


In [7]:
# add columns for data understandability in plots
def get_file_size_mb(path):
    """
    Returns the size in MB of a file or total size of all files in a directory.
    """
    p = pathlib.Path(path)
    try:
        if p.is_file():
            return p.stat().st_size / 1024 / 1024
        elif p.is_dir():
            return (
                sum(f.stat().st_size for f in p.rglob("*") if f.is_file()) / 1024 / 1024
            )
        else:
            return None
    except FileNotFoundError:
        return None


# memory usage in MB
df_results["peak_memory (GB)"] = df_results["peak_memory (bytes)"] / 1024 / 1024 / 1024

# data input size additions for greater context
df_results["data_input_size_mb"] = df_results["data_input"].apply(get_file_size_mb)
df_results["data_input_with_size"] = (
    df_results["data_input"]
    + " ("
    + round(df_results["data_input_size_mb"]).astype("int64").astype("str")
    + " MB)"
)

# rename data input to simplify
df_results["data_input_renamed"] = (
    df_results["data_input_with_size"]
    .str.replace(f"{examples_dir}/data/", "")
    .str.replace("examplehuman_cellprofiler_features_", "input_")
)
df_results

,file_input,data_input,iteration,time_duration (secs),peak_memory (bytes),peak_memory (GB),data_input_size_mb,data_input_with_size,data_input_renamed
0,cytotable_convert_examplehuman_multiprocess_cs...,examples/data/examplehuman_cellprofiler_featur...,0,20.981794,3.349709e+08,0.311966,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
1,cytotable_convert_examplehuman_multiprocess_cs...,examples/data/examplehuman_cellprofiler_featur...,1,20.731451,3.349135e+08,0.311913,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
2,cytotable_convert_examplehuman_multiprocess_cs...,examples/data/examplehuman_cellprofiler_featur...,2,20.990158,3.348234e+08,0.311829,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
3,cytotable_convert_examplehuman_multiprocess_cs...,examples/data/examplehuman_cellprofiler_featur...,3,20.725288,3.350774e+08,0.312065,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
4,cytotable_convert_examplehuman_multiprocess_cs...,examples/data/examplehuman_cellprofiler_featur...,4,20.749313,3.342705e+08,0.311314,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB)
...,...,...,...,...,...,...,...,...,...
247,pandas_merge_examplehuman_csv.py,examples/data/examplehuman_cellprofiler_featur...,1,95.369457,2.429664e+10,22.628010,10922.421144,examples/data/examplehuman_cellprofiler_featur...,input_csv-x8192 (10922 MB)
248,pandas_merge_examplehuman_csv.py,examples/data/examplehuman_cellprofiler_featur...,2,92.656957,2.468546e+10,22.990124,10922.421144,examples/data/examplehuman_cellprofiler_featur...,input_csv-x8192 (10922 MB)
249,pandas_merge_examplehuman_csv.py,examples/data/examplehuman_cellprofiler_featur...,3,92.837449,2.389194e+10,22.251102,10922.421144,examples/data/examplehuman_cellprofiler_featur...,input_csv-x8192 (10922 MB)
250,pandas_merge_examplehuman_csv.py,examples/data/examplehuman_cellprofiler_featur...,4,92.357122,2.406183e+10,22.409325,10922.421144,examples/data/examplehuman_cellprofiler_featur...,input_csv-x8192 (10922 MB)


In [8]:
# build cols for split reference in the plot
df_results["cytotable_time_duration (multiprocess) (secs)"] = df_results[
    df_results["file_input"] == "cytotable_convert_examplehuman_multiprocess_csv.py"
]["time_duration (secs)"]
df_results["cytotable_peak_memory (multiprocess) (GB)"] = df_results[
    df_results["file_input"] == "cytotable_convert_examplehuman_multiprocess_csv.py"
]["peak_memory (GB)"]
df_results["cytotable_time_duration (multithread) (secs)"] = df_results[
    df_results["file_input"] == "cytotable_convert_examplehuman_multithread_csv.py"
]["time_duration (secs)"]
df_results["cytotable_peak_memory (multithread) (GB)"] = df_results[
    df_results["file_input"] == "cytotable_convert_examplehuman_multithread_csv.py"
]["peak_memory (GB)"]
df_results["pandas_time_duration (secs)"] = df_results[
    df_results["file_input"] == "pandas_merge_examplehuman_csv.py"
]["time_duration (secs)"]
df_results["pandas_peak_memory (GB)"] = df_results[
    df_results["file_input"] == "pandas_merge_examplehuman_csv.py"
]["peak_memory (GB)"]
df_results = (
    df_results.apply(lambda x: pd.Series(x.dropna().values))
    .drop(["file_input", "time_duration (secs)", "peak_memory (GB)"], axis=1)
    .dropna()
)
df_results

,data_input,iteration,peak_memory (bytes),data_input_size_mb,data_input_with_size,data_input_renamed,cytotable_time_duration (multiprocess) (secs),cytotable_peak_memory (multiprocess) (GB),cytotable_time_duration (multithread) (secs),cytotable_peak_memory (multithread) (GB),pandas_time_duration (secs),pandas_peak_memory (GB)
0,examples/data/examplehuman_cellprofiler_featur...,0,3.349709e+08,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB),20.981794,0.311966,2.997917,0.243740,0.702126,0.110638
1,examples/data/examplehuman_cellprofiler_featur...,1,3.349135e+08,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB),20.731451,0.311913,2.991671,0.248013,0.584849,0.114052
2,examples/data/examplehuman_cellprofiler_featur...,2,3.348234e+08,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB),20.990158,0.311829,3.009408,0.235371,0.695516,0.112953
3,examples/data/examplehuman_cellprofiler_featur...,3,3.350774e+08,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB),20.725288,0.312065,2.998291,0.235367,0.701317,0.112778
4,examples/data/examplehuman_cellprofiler_featur...,4,3.342705e+08,1.414125,examples/data/examplehuman_cellprofiler_featur...,input_csv (1 MB),20.749313,0.311314,2.941357,0.235741,0.704329,0.112957
...,...,...,...,...,...,...,...,...,...,...,...,...
79,examples/data/examplehuman_cellprofiler_featur...,1,2.767725e+09,10922.421144,examples/data/examplehuman_cellprofiler_featur...,input_csv-x8192 (10922 MB),141.259489,2.577644,141.305364,4.570572,95.369457,22.628010
80,examples/data/examplehuman_cellprofiler_featur...,2,2.768441e+09,10922.421144,examples/data/examplehuman_cellprofiler_featur...,input_csv-x8192 (10922 MB),141.573577,2.578312,141.732157,4.630157,92.656957,22.990124
81,examples/data/examplehuman_cellprofiler_featur...,3,2.724692e+09,10922.421144,examples/data/examplehuman_cellprofiler_featur...,input_csv-x8192 (10922 MB),142.025290,2.537567,141.402291,4.601284,92.837449,22.251102
82,examples/data/examplehuman_cellprofiler_featur...,4,2.800398e+09,10922.421144,examples/data/examplehuman_cellprofiler_featur...,input_csv-x8192 (10922 MB),141.646941,2.608074,141.124171,4.666851,92.357122,22.409325


In [9]:
# Group by data_input_renamed and calculate mean, min, and max
aggregated_results = df_results.groupby("data_input_renamed").agg(
    {
        "cytotable_time_duration (multiprocess) (secs)": ["mean", "min", "max"],
        "cytotable_time_duration (multithread) (secs)": ["mean", "min", "max"],
        "pandas_time_duration (secs)": ["mean", "min", "max"],
        "cytotable_peak_memory (multiprocess) (GB)": ["mean", "min", "max"],
        "cytotable_peak_memory (multithread) (GB)": ["mean", "min", "max"],
        "pandas_peak_memory (GB)": ["mean", "min", "max"],
    }
)
# Flatten the multi-level columns
aggregated_results.columns = [
    f"{col[0]} ({col[1]})" for col in aggregated_results.columns
]
aggregated_results.reset_index(inplace=True)


# Helper function to extract numeric value or None
def sort_key(s):
    match = re.search(r"\d+", s)
    if match:
        return (1, int(match.group()))  # numeric items: (1, number)
    else:
        return (0, s.lower())  # non-numeric items: (0, alphabetical)


# Sort using the custom key
aggregated_results = aggregated_results.sort_values(
    by="data_input_renamed", key=lambda col: col.map(sort_key)
)
aggregated_results

,data_input_renamed,cytotable_time_duration (multiprocess) (secs) (mean),cytotable_time_duration (multiprocess) (secs) (min),cytotable_time_duration (multiprocess) (secs) (max),cytotable_time_duration (multithread) (secs) (mean),cytotable_time_duration (multithread) (secs) (min),cytotable_time_duration (multithread) (secs) (max),pandas_time_duration (secs) (mean),pandas_time_duration (secs) (min),pandas_time_duration (secs) (max),cytotable_peak_memory (multiprocess) (GB) (mean),cytotable_peak_memory (multiprocess) (GB) (min),cytotable_peak_memory (multiprocess) (GB) (max),cytotable_peak_memory (multithread) (GB) (mean),cytotable_peak_memory (multithread) (GB) (min),cytotable_peak_memory (multithread) (GB) (max),pandas_peak_memory (GB) (mean),pandas_peak_memory (GB) (min),pandas_peak_memory (GB) (max)
0,input_csv (1 MB),20.833497,20.725288,20.990158,2.979258,2.936904,3.009408,0.680773,0.584849,0.704329,0.311885,0.311314,0.312222,0.238688,0.233894,0.248013,0.112738,0.110638,0.114052
4,input_csv-x2 (3 MB),20.964547,20.640574,21.082795,3.176469,3.054220,3.243892,0.698571,0.675399,0.711165,0.312931,0.312225,0.313965,0.246026,0.236958,0.257481,0.113965,0.110260,0.115913
8,input_csv-x4 (5 MB),21.097768,20.837013,21.212725,3.524618,3.487053,3.554939,0.703272,0.701723,0.708285,0.314633,0.313847,0.315407,0.245380,0.237988,0.273193,0.118576,0.118336,0.118725
12,input_csv-x8 (11 MB),21.563956,21.441576,21.738663,4.149511,4.091795,4.218827,0.720569,0.701735,0.801708,0.321948,0.317844,0.339405,0.262403,0.242653,0.281143,0.126362,0.126137,0.126438
3,input_csv-x16 (21 MB),21.824366,21.718493,21.920061,4.598993,4.526463,4.638726,0.846868,0.807117,0.907765,0.341398,0.339573,0.344280,0.287287,0.262241,0.318546,0.142043,0.141747,0.142303
7,input_csv-x32 (43 MB),22.309466,21.711435,22.702538,5.436012,5.382754,5.496539,1.051788,1.032278,1.068415,0.403185,0.358887,0.491158,0.350104,0.331715,0.369049,0.174838,0.171955,0.175629
11,input_csv-x64 (85 MB),23.376563,23.229440,23.452713,7.031577,6.958911,7.126724,1.444839,1.402900,1.473439,0.538514,0.496964,0.585533,0.498933,0.481983,0.540123,0.239684,0.237415,0.250610
2,input_csv-x128 (170 MB),24.227756,24.126119,24.319255,8.203636,8.114115,8.305845,2.179335,2.100766,2.224394,0.711216,0.609779,0.779457,0.692265,0.631813,0.730137,0.378014,0.347889,0.416374
6,input_csv-x256 (341 MB),25.713034,25.490880,25.904561,10.041020,9.956017,10.140570,3.798717,3.717893,3.848830,1.029647,0.857513,1.086540,1.045956,0.911758,1.174927,0.603293,0.572678,0.629745
10,input_csv-x512 (682 MB),29.012794,28.721548,29.527593,13.390294,13.281820,13.477414,6.778361,6.694391,6.884983,1.490635,1.411068,1.626228,1.717751,1.649502,1.774868,1.179618,1.067192,1.271629


In [10]:
# Time plot with min and max errors
fig = px.line(
    aggregated_results,
    x="data_input_renamed",
    y=[
        "cytotable_time_duration (multiprocess) (secs) (mean)",
        "cytotable_time_duration (multithread) (secs) (mean)",
        "pandas_time_duration (secs) (mean)",
    ],
    title="CytoTable and Pandas CSV Processing Time with Min/Max Errors",
    labels={"data_input_renamed": "Input File", "value": "Seconds"},
    height=500,
    width=900,
    symbol_sequence=["diamond"],
    color_discrete_sequence=[
        px.colors.qualitative.Vivid[6],
        px.colors.qualitative.Vivid[7],
        px.colors.qualitative.Vivid[4],
    ],
)

# Add error bars for each trace
for i, col in enumerate(
    [
        "cytotable_time_duration (multiprocess) (secs)",
        "cytotable_time_duration (multithread) (secs)",
        "pandas_time_duration (secs)",
    ]
):
    fig.data[i].update(
        error_y=dict(
            array=(
                aggregated_results[f"{col} (max)"] - aggregated_results[f"{col} (mean)"]
            ),
            arrayminus=(
                aggregated_results[f"{col} (mean)"] - aggregated_results[f"{col} (min)"]
            ),
        )
    )

# Rename the lines for the legend
newnames = {
    "cytotable_time_duration (multiprocess) (secs) (mean)": "CytoTable (multiprocess)",
    "cytotable_time_duration (multithread) (secs) (mean)": "CytoTable (multithread)",
    "pandas_time_duration (secs) (mean)": "Pandas",
}
fig.for_each_trace(
    lambda t: t.update(
        name=newnames[t.name],
        legendgroup=newnames[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name]),
    )
)

# Update the legend and layout
fig.update_layout(
    legend_title_text="",
    legend=dict(x=0.01, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
    font=dict(size=16),  # Global font size
)
fig.update_traces(mode="lines+markers")
fig.update_layout(yaxis=dict(rangemode="tozero", autorange=True))

# Save and display the plot
fig.write_image(join_read_time_image)
fig.write_image(join_read_time_image.replace(".png", ".svg"))
Image(url=join_read_time_image.replace(".png", ".svg"))

In [11]:
# Memory plot with min and max errors
fig = px.line(
    aggregated_results,
    x="data_input_renamed",
    y=[
        "cytotable_peak_memory (multiprocess) (GB) (mean)",
        "cytotable_peak_memory (multithread) (GB) (mean)",
        "pandas_peak_memory (GB) (mean)",
    ],
    title="CytoTable and Pandas CSV Peak Memory with Min/Max Errors",
    labels={"data_input_renamed": "Input File", "value": "Gigabytes (GB)"},
    height=500,
    width=900,
    symbol_sequence=["diamond"],
    color_discrete_sequence=[
        px.colors.qualitative.Vivid[6],
        px.colors.qualitative.Vivid[7],
        px.colors.qualitative.Vivid[4],
    ],
)

# Add error bars for each trace
for i, col in enumerate(
    [
        "cytotable_peak_memory (multiprocess) (GB)",
        "cytotable_peak_memory (multithread) (GB)",
        "pandas_peak_memory (GB)",
    ]
):
    fig.data[i].update(
        error_y=dict(
            array=(
                aggregated_results[f"{col} (max)"] - aggregated_results[f"{col} (mean)"]
            ),
            arrayminus=(
                aggregated_results[f"{col} (mean)"] - aggregated_results[f"{col} (min)"]
            ),
        )
    )

# Rename the lines for the legend
newnames = {
    "cytotable_peak_memory (multiprocess) (GB) (mean)": "CytoTable (multiprocess)",
    "cytotable_peak_memory (multithread) (GB) (mean)": "CytoTable (multithread)",
    "pandas_peak_memory (GB) (mean)": "Pandas",
}
fig.for_each_trace(
    lambda t: t.update(
        name=newnames[t.name],
        legendgroup=newnames[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name]),
    )
)

# Update the legend and layout
fig.update_layout(
    legend_title_text="",
    legend=dict(x=0.01, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
    font=dict(size=16),  # Global font size
)
fig.update_traces(mode="lines+markers")
fig.update_layout(yaxis=dict(rangemode="tozero", autorange=True))

# Save and display the plot
fig.write_image(join_mem_size_image)
fig.write_image(join_mem_size_image.replace(".png", ".svg"))
Image(url=join_mem_size_image.replace(".png", ".svg"))